In [65]:
import trimesh
import numpy as np
import k3d

import polars as pl

from pathlib import Path

import os
import glob
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm



In [66]:
df = pl.read_csv("ShapenetSem/metadata.csv")


categories = "(?i)bed|chair|couch|desk|table|lamp|shelf|wardrobe"
result_df = df.filter(
    (pl.col("category").str.contains(categories)) & 
    (pl.col("unit").is_not_null())   # skip models without units
)


result_df

fullId,category,wnsynset,wnlemmas,up,front,unit,aligned.dims,isContainerLike,surfaceVolume,solidVolume,supportSurfaceArea,weight,staticFrictionForce,name,tags
str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str
"""wss.100f39dce7690f59efb94709f3…","""Chair,Recliner""","""n4069540""","""recliner,reclining chair,loung…","""0\,0\,1""","""0\,-1\,0""",0.012947,"""111.34567\,100.547745\,96.1327…",null,null,null,null,null,null,"""couch""","""carpet recliner"""
"""wss.1022fe7dd03f6a4d4d5ad9f13a…","""Chair,OfficeChair""","""n3005231""","""chair""","""0\,0\,1""","""0\,1\,0""",0.017984,"""60.366123\,98.00925\,66.79712""",null,null,null,null,null,null,"""office chair""","""fauteuil de bureau,office chai…"
"""wss.1028b32dc1873c2afe26a3ac36…","""Chair,OfficeSideChair""","""n3005231""","""chair""","""0\,0\,1""","""0\,-1\,0""",0.036115,"""78.41001\,117.685616\,77.54468""",null,null,null,null,null,null,"""visavis stackable chair""","""chair office designer citterio"""
"""wss.102a6b7809f4e51813842bc8ef…","""Computer,Desktop""","""n3184677""","""desktop computer""",null,null,0.025252,"""23.257397\,46.2361\,60.778683""",null,null,null,null,null,null,"""alienware area computer cas…","""*"""
"""wss.1033ee86cc8bac4390962e4fb7…","""Chair""","""n2741540""","""armchair""","""0\,0\,1""","""-1\,0\,0""",0.024691,"""66.430275\,98.00357\,109.57538""",null,null,null,null,null,null,"""the armchair cit eacute""","""armchair,chair,chaise,french,f…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""wss.f69510412f10e9ed322ed2ef5f…","""Lamp,WallLamp""","""n4155768""","""sconce""","""0\,0\,1""","""-1\,0\,0""",0.059747,"""42.131245\,90.37565\,36.08628""",null,null,null,null,null,null,"""antique lighting sconce""","""antique,double e,light,old,sco…"
"""wss.738cebb6eab9151bd4a8d9e46a…","""Lamp,WallLamp""","""n3670692""","""light,light source""","""0\,0\,1""","""1\,0\,0""",0.08408,"""51.64453\,84.50152\,37.83612""",null,null,null,null,null,null,"""vaporproof wall light jelly j…","""jelly jar,light,vapor,vaporpro…"
"""wss.571a3c2cf4c509596cb5b29312…","""Lamp,WallLamp""","""n4155768""","""sconce""","""0\,1\,0""","""0\,0\,1""",0.069445,"""69.44497\,39.93086\,69.44497""",null,null,null,null,null,null,"""sconce round""","""contemporary,lamp,light,lighti…"


In [68]:
model_path = Path("ShapenetSem/models-COLLADA/COLLADA/")

# Create k3d plot with empty mesh and label
plot = k3d.plot(camera_mode="orbit", grid_auto_fit=False, grid=[-2, -2, 0, 2, 2, 2])
mesh_obj = k3d.mesh(np.zeros((0, 3), dtype=np.float32), np.zeros((0, 3), dtype=np.uint32), color=0x00FF00)
label = k3d.text2d("", position=(0.1, 0.1), size=1.0, color=0x0)
plot += mesh_obj
plot += label
plot.display()

def update_mesh(index):
    """Update the displayed mesh based on slider index."""
    row = result_df.row(index, named=True)
    fullid = row["fullId"][4:]
    mesh_file = model_path / f"{fullid}.dae"
    
    try:
        mesh = trimesh.load_mesh(mesh_file)
        
        # Apply unit scale if available
        if row['unit'] is not None:
            mesh.apply_scale(float(row['unit']))
        
        mesh_obj.vertices = mesh.vertices.astype(np.float32)
        mesh_obj.indices = mesh.faces.astype(np.uint32)
        
        up_val = row['up'] if row['up'] else 'N/A'
        front_val = row['front'] if row['front'] else 'N/A'
        label.text = f"up: {up_val}  |  front: {front_val}"
        
        print(f"Model {index}: {row['name']} ({row['category']})")
    except Exception as e:
        print(f"Failed to load {fullid}: {e}")

slider = widgets.IntSlider(
    value=0, min=0, max=len(result_df) - 1, step=1,
    description='Model:', continuous_update=False,
    layout=widgets.Layout(width='100%')
)

widgets.interact(update_mesh, index=slider)

Output()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Model:', layout=Layout(width='1…

<function __main__.update_mesh(index)>

In [42]:
# get t

categories = "(?i)bed|chair|couch|desk|table|lamp|shelf|wardrobe"
result_df = df.filter(
    (pl.col("category").str.contains(categories)) & 
    (pl.col("unit").is_not_null()) &   # skip models without units
    (pl.col("up").is_not_null())   # skip models without units
    
)

model_path = Path("ShapenetSem/models-COLLADA/COLLADA/")

row = result_df.filter(pl.col("up") == "1\\,0\\,0").head(1)
print(row)
fullid = row["fullId"].item()[4:]

print(f"Found model: {fullid}")
print(model_path / f"{fullid}.dae")

mesh = trimesh.load_mesh(model_path / f"{fullid}.dae")
mesh.show()

shape: (0, 16)
┌────────┬──────────┬──────────┬──────────┬───┬────────┬─────────────────────┬──────┬──────┐
│ fullId ┆ category ┆ wnsynset ┆ wnlemmas ┆ … ┆ weight ┆ staticFrictionForce ┆ name ┆ tags │
│ ---    ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---    ┆ ---                 ┆ ---  ┆ ---  │
│ str    ┆ str      ┆ str      ┆ str      ┆   ┆ str    ┆ str                 ┆ str  ┆ str  │
╞════════╪══════════╪══════════╪══════════╪═══╪════════╪═════════════════════╪══════╪══════╡
└────────┴──────────┴──────────┴──────────┴───┴────────┴─────────────────────┴──────┴──────┘


ValueError: can only call '.item()' if the Series is of length 1, or an explicit index is provided (Series is of length 0)

In [6]:
# filter categories



categories = "(?i)bed|chair|couch|desk|table|lamp|shelf|wardrobe"
result_df = df.filter(
    (pl.col("category").str.contains(categories)) & 
    (pl.col("unit").is_not_null())   # skip models without units
)

output_df = result_df.select([
    # 1. Remove first 4 characters from fullId
    pl.col("fullId").str.slice(4),
    
    # 2. Process category column
    pl.col("category")
    .str.split(",")                           # Split into a list of words
    .list.eval(                               # Evaluate each item in the list
        pl.element().filter(
            # Keep elements that do NOT start with "_" (after trimming spaces)
            ~pl.element().str.strip_chars().str.starts_with("_")
        )
    )
    .list.join(",")                           # Join the remaining words back
])

# Write to CSV
output_df.write_csv("inventory.csv")

# Optional: Print to verify
print(output_df.head())

shape: (5, 2)
┌─────────────────────────────────┬───────────────────────┐
│ fullId                          ┆ category              │
│ ---                             ┆ ---                   │
│ str                             ┆ str                   │
╞═════════════════════════════════╪═══════════════════════╡
│ 100f39dce7690f59efb94709f30ce0… ┆ Chair,Recliner        │
│ 1022fe7dd03f6a4d4d5ad9f13ac9f4… ┆ Chair,OfficeChair     │
│ 1028b32dc1873c2afe26a3ac360dbd… ┆ Chair,OfficeSideChair │
│ 102a6b7809f4e51813842bc8ef6fe1… ┆ Computer,Desktop      │
│ 1033ee86cc8bac4390962e4fb7072b… ┆ Chair                 │
└─────────────────────────────────┴───────────────────────┘


In [ ]:
# convert the collada files to glb 
# we work with result_df

model_path = Path("ShapenetSem/models-COLLADA/")
glb_path = Path("GLB/")
glb_path.mkdir(exist_ok=True, parents=True) # Ensure output directory exists


for row in tqdm(result_df.iter_rows(named=True), total=len(result_df)):
    full_id = row["fullId"]
    unit = row["unit"]
    
    # Skip if unit is None/Null to avoid errors (should not happen)
    if unit is None: 
        print(f"Skipping {full_id}: No unit defined")
        continue

    # 1. Prepare ID and paths
    short_id = full_id[4:]  # Remove first 4 chars (wss.)
    source_file = model_path / f"{short_id}.dae"
    dest_file = glb_path / f"{short_id}.glb"

    try:
        tm_mesh = trimesh.load(source_file, force='mesh')
        tm_mesh.apply_scale(float(unit))
        tm_mesh.export(dest_file)
        
    except Exception as e:
        print(f"Failed to process {short_id}: {e}")

print("Processing complete.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [03:39<00:00, 12.67it/s]

Processing complete.


In [41]:
# test in notebook with k3d and trimesh

{
  "text": "{\"thread_id\":\"f00384b4-fd30-4a10-899f-de400cd24b5b\",\"query\":\"dining table\",\"citations\":[{\"id\":\"56381524-a744-4fdf-a54f-5887e60228a8\",\"text\":\"d0176f9a144100cd, Chair\\nc9dd71db5e6713937d0fed3563de13d, TableClock,Clock\\nc9ed0aa479b6859340cb4688019498aa, Chair,Chaise\\nca000b9d32450bc4eb6d0fd624ec3fc, Table,DiningTable\\nca0cb42682d0635abec3353c29751fdf, Lamp,CeilingLamp\\nca19060096ee6223d54ba5ae13a4d04c, CeilingLamp\\nca3e6e011ed7ecb32dc86554a873c4fe, Couch\\nca471ae192f76fcd322ed2ef5fc90e25, Lamp,CeilingLamp\\nca51779877894f6217d272ea353a61a0, Couch,Loveseat\\nca56e6241a3c762a391c070c9832629, Table,DiningTable\\nca6c07357ba5125b8e2adb29857f8a1, Table,DiningTable,R\",\"score\":0.44777834,\"file_id\":\"6bf4fdb2-f063-4231-935d-03a47d213f5e\",\"chunk_file_order\":210},{\"id\":\"cd2b53e8-940b-4582-9a01-28b0ccec84f6\",\"text\":\"3f, Table,RoundTable\\n1361850926bce33f84e64ea9646404ec, Lamp,FloorLamp\\n1383d18af610241f69d8d5b35c38db0d, Chair,SideChair\\n13a4536a8c7dbc87e3f7a74e12a274ef, Table,RoundTable\\n13aa33650c706094732375ced502498, Chair\\n13b365ef4887f5d388a942de43574033, Lamp,CeilingLamp\\n13c18609602e4ced37b2bb75885cfc44, Chair,ChairWithOttoman\\n13c8bf6b41e555b78821ad3452f1c8c9, Lamp,LampPost\\n13dbeeacdabce3b694658a0201ba0367, ChestOfDrawers,Armoire,Dresser,Wardrobe\\n13f565d814bd5647adcda92177beb1f7, ChestOfDrawers,Armoire,Wardrobe\\n142f1b\",\"score\":0.44450042,\"file_id\":\"6bf4fdb2-f063-4231-935d-03a47d213f5e\",\"chunk_file_order\":4},{\"id\":\"5e8ae460-2692-4dc1-abbb-cd8e76140a8f\",\"text\":\"ble\\n4b384532c11edd13f34d94e94d16f9e4, Computer,Desktop\\n4b92aa95cc2210c3f017dd57a82a3a39, Table,DiningTable,RoundTable\\n4bef120ccf16226eb0aa15078ea6f391, Lamp,DeskLamp\\n4c6ab617f0d35e37da2e1014c2799f3b, Lamp,TableLamp\\n4e222d6b128a7af7b8fa4cb65e077db1, Chair,AccentChair\\n4e3d1346e76084a2c1caa73a5e644b6, Lamp,FloorLamp\\n4e83d107a17dd89ae3f7a74e12a274ef, Bed,QueenBed\\n4f3b404cc30d2306b96ac7572d4197cf, Table,DiningTable,RoundTable\\n5008d5e0ddf1b3dfa37d17ad9e86b6bb, Chair\\n534a6db0833bbb89504721639e19f609, Chair,SideCha\",\"score\":0.4360456,\"file_id\":\"6bf4fdb2-f063-4231-935d-03a47d213f5e\",\"chunk_file_order\":273}],\"mode\":\"auto\"}",
  "type": "text"
}

# load an obj and visualize with trimesh 

mesh = trimesh.load_mesh("GLB/107637b6bdf8129d4904d89e9169817b.glb")
k3d_mesh = k3d.mesh(
    mesh.vertices.astype(np.float32), 
    mesh.faces.astype(np.uint32),
    color=0x00FF00,  # Hex color (Green)
    wireframe=False  # Set to True to see edges
)

# 3. Instantiate the plot and display
plot = k3d.plot(camera_mode="orbit")
plot += k3d_mesh
plot.display()

/Users/d.p.saakes/code/invoke-xr/.venv/lib/python3.12/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/d.p.saakes/code/invoke-xr/.venv/lib/python3.12/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()